In [1]:
import numpy as np
import pandas as pd

In [15]:
users = pd.read_csv('dataset/MovieLens/u.user', sep='|', header=None, names=['user_id', 'age', 'gender', 'job', 'zip-code'])
users['gender'] = users['gender'].map({'M': -1, 'F': 1})
users['one'] = 1
users = users.pivot_table(index=['user_id', 'age', 'gender'], columns='job', values='one', fill_value=0)
users = users.reset_index()
age_bins = [10 * i for i in range(1, 10)]
age_labels = [f'age-{x}s' for x in age_bins[:-1]]
users['age_bin'] = pd.cut(users['age'], bins=age_bins, labels=age_labels)
agg_cols = [col for col in users.columns if col not in 'age_bin']
users['one'] = 1
age = users.pivot_table(index='user_id', columns='age_bin', values='one', fill_value=0)
users = users.merge(age, left_on='user_id', right_index=True)
users = users.drop('one', axis=1)
users['user_id'] -= 1
users['age_std'] = (users['age'] - np.mean(users['age'])) / np.std(users['age']) / 2
users = users.drop(['age_bin', 'age'], axis=1)
users = users.rename({'user_id': 'uid'}, axis=1)
users.head()

,uid,gender,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,...,technician,writer,age-10s,age-20s,age-30s,age-40s,age-50s,age-60s,age-70s,age_std
0,0,-1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,-0.416174
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.778039
2,2,-1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,-0.457353
3,3,-1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,-0.416174
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,-0.045556


In [24]:
genres = pd.read_csv('dataset/MovieLens/u.genre', sep='|', header=None, names=['genre', 'genre_id'])
movies_columns = tuple(['movie_id', 'title', 'date', 'empty', 'url'] + list(genres['genre'].values))
movies = pd.read_csv('dataset/MovieLens/u.item', sep='|', header=None, index_col=False,
                     names=movies_columns, encoding='ISO-8859-1')
movies = movies.drop(['date', 'empty', 'url', 'unknown'], axis=1)
movies['movie_id'] -= 1
movies.columns = map(str.lower, movies.columns)
movies = movies.rename({'movie_id': 'iid'}, axis=1)
movies.head()

,iid,title,action,adventure,animation,children's,comedy,crime,documentary,drama,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,0,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,3,Get Shorty (1995),1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,4,Copycat (1995),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [27]:
ratings = pd.read_csv('dataset/MovieLens/u.data', sep='\t', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
ratings['user_id'] -= 1
ratings['movie_id'] -= 1
ratings = ratings.rename({'movie_id': 'iid', 'user_id': 'uid'}, axis=1)
ratings.head()

,uid,iid,rating,timestamp
0,195,241,3,881250949
1,185,301,3,891717742
2,21,376,1,878887116
3,243,50,2,880606923
4,165,345,1,886397596


In [29]:
for df, out_name in zip((users, movies, ratings), ('users', 'items', 'ratings')):
    df.to_csv(f'dataset/MovieLens/{out_name}.csv', index=False)